In [1]:
import polars as pl

In [ ]:
def stock_investment_monthly(
    initial_investment: float,
    monthly_contribution: float,
    annual_return: float,
    years: int,
    annual_inflation: float = 0.0,
) -> pl.DataFrame:
    n_months = years * 12
    r_monthly = annual_return / 12.0
    i_monthly = annual_inflation / 12.0

    balance = [initial_investment]
    contributions_cum = [initial_investment]
    returns_cum = [0.0]

    for m in range(1, n_months + 1):
        # previous balance grows
        interest = balance[-1] * r_monthly
        new_balance = balance[-1] + interest
        # add monthly contribution
        new_balance += monthly_contribution

        # update cumulative trackers
        balance.append(new_balance)
        contributions_cum.append(contributions_cum[-1] + monthly_contribution)
        returns_cum.append(returns_cum[-1] + interest)

    df = pl.DataFrame(
        {
            "month": [int(m) for m in range(n_months + 1)],
            "year": [int(m // 12) for m in range(n_months + 1)],
            "balance": [float(b) for b in balance],
            "contributions_cum": [float(c) for c in contributions_cum],
            "returns_cum": [float(r) for r in returns_cum],
        }
    )

    df = df.with_columns(
        [
            (pl.col("balance") / ((1 + i_monthly) ** pl.col("month"))).alias(
                "balance_real"
            ),
            (pl.col("contributions_cum") / ((1 + i_monthly) ** pl.col("month"))).alias(
                "contributions_real"
            ),
        ]
    )
    return df

In [11]:
stock_investment_monthly(
    initial_investment=15000,
    monthly_contribution=15000,
    annual_return=0.07,
    years=10,
    annual_inflation=0.02,
)

month,year,balance,contributions_cum,returns_cum,balance_real,contributions_real
i64,i64,f64,f64,f64,f64,f64
0,0,15000.0,15000.0,0.0,15000.0,15000.0
1,0,30087.5,30000.0,87.5,30037.437604,29950.083195
2,0,45263.010417,45000.0,263.010417,45112.510071,44850.374168
3,0,60527.044644,60000.0,527.044644,60225.41541,59700.997229
4,0,75880.119071,75000.0,880.119071,75376.352386,74502.076409
…,…,…,…,…,…,…
116,9,2.5069e6,1.755e6,751882.253155,2.0665e6,1.4467e6
117,9,2.5365e6,1.77e6,766505.732965,2.0875e6,1.4567e6
118,9,2.5663e6,1.785e6,781302.016407,2.1085e6,1.4666e6
